In [1]:
import numpy as np
import cv2
import torch 
import torchvision 
from engine import train_one_epoch, evaluate
from data_reading import WindowDataset
import utils
from torch.utils.data import DataLoader

In [2]:
def get_model():
    # Initialize Model, from pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    num_classes = 2 # window + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features # Number of input features for the classifier
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model.double()

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on device {device}')
model = get_model()

c:\Users\pchen23\Desktop\FasterRCNN-Baseline\.env\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 8000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


running on device cpu


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\pchen23/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100.0%


In [4]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr = 0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [5]:
train = DataLoader(WindowDataset(), batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
print(next(iter(train))[0])
print(next(iter(train))[1])

(tensor([[[204., 201., 208.,  ..., 209., 209., 209.],
         [203., 202., 208.,  ..., 209., 209., 209.],
         [202., 204., 209.,  ..., 210., 210., 210.],
         ...,
         [225., 233., 235.,  ..., 213., 229., 245.],
         [225., 231., 232.,  ..., 207., 224., 242.],
         [224., 231., 231.,  ..., 203., 221., 239.]],

        [[229., 226., 233.,  ..., 251., 251., 251.],
         [228., 227., 233.,  ..., 251., 251., 251.],
         [227., 229., 234.,  ..., 250., 250., 250.],
         ...,
         [245., 253., 254.,  ..., 199., 218., 234.],
         [246., 252., 253.,  ..., 193., 213., 231.],
         [245., 252., 252.,  ..., 189., 210., 228.]],

        [[226., 223., 230.,  ..., 241., 241., 241.],
         [225., 224., 230.,  ..., 241., 241., 241.],
         [224., 226., 231.,  ..., 241., 241., 241.],
         ...,
         [234., 244., 248.,  ..., 196., 214., 230.],
         [237., 243., 246.,  ..., 190., 209., 227.],
         [236., 243., 245.,  ..., 186., 206., 224.]]

In [6]:
print(next(iter(train))[1][0]['labels'].dtype)

torch.int64


In [7]:
imgs, labs = next(iter(train))
model.eval()
model(imgs)

[{'boxes': tensor([[  0.0000,   0.0000, 128.0000,  18.5000],
          [ 25.3439,  94.0413,  28.8117,  94.6941],
          [ 56.9590,  93.9772,  62.3299,  93.9881],
          [ 80.7615,  57.6415,  80.7757,  58.2784],
          [111.5747,  58.6350, 111.6006,  59.0576],
          [ 60.2666,  56.3064,  60.2730,  57.2669],
          [ 56.3777,  55.4562,  56.3868,  56.5525],
          [ 70.0787,  55.4686,  70.1054,  56.2004],
          [ 72.9064,  56.9892,  72.9149,  57.7352],
          [ 81.7831,  55.7227,  81.8255,  56.2753],
          [ 90.2068,   0.0000,  90.2084,  96.0000],
          [ 84.0543,  56.1921,  84.1069,  56.4128],
          [ 71.5838,   0.0000,  71.8912,  96.0000],
          [113.0667,   2.3328, 113.3931,  96.0000],
          [ 97.9625,   0.0000,  98.2504,  96.0000]], dtype=torch.float64,
         grad_fn=<StackBackward>),
  'labels': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  'scores': tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0

In [10]:
num_epochs = 40

for epoch in range(num_epochs):
    # Train for one epoch, while printing every 10 iterations
    train_one_epoch(model, optimizer, train, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, train, device)

print("DONE")

({'boxes': tensor([[110.,  57., 127.,  95.]]), 'labels': tensor([1])},)
Epoch: [0]  [   0/1029]  eta: 7:42:03  lr: 0.000010  loss: 0.8136 (0.8136)  loss_classifier: 0.1206 (0.1206)  loss_box_reg: 0.4247 (0.4247)  loss_objectness: 0.2573 (0.2573)  loss_rpn_box_reg: 0.0111 (0.0111)  time: 26.9426  data: 0.0020
({'boxes': tensor([[ 64.,  48., 127.,  95.]]), 'labels': tensor([1])},)
({'boxes': tensor([[ 68.,  48., 127.,  95.]]), 'labels': tensor([1])},)
({'boxes': tensor([[ 64.,  48., 127.,  95.]]), 'labels': tensor([1])},)


KeyboardInterrupt: 